In [1]:
import torch
from torch import nn
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
from smokingml.datasets.nursing_dataset_v1 import WINSIZE, NursingDatasetV1, nursingv1_train_dev_test_split
from smokingml.datasets.nursing_dataset_v1.dataloading import load_one_session, load_one_windowed_session
from smokingml.datasets.nursing_dataset_v1.utils import window_session, train_test_split_windows, get_all_session_ids
from smokingml.modules import train_loop, optimization_loop, evaluate_loop

In [2]:
data_dir = Path('../data/nursingv1_dataset/')
device = 'cuda:1'

# 1st LSTM: One Sequence of Points
- entire session is sequence of inputs
- each input is one data point (xyz)
- input shape to model is ($L$,$H_{in}$)

$L =$ `len(session)`

$H_{in} = 3$

In [13]:
class LSTM1(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=3, hidden_size=64, bias=False)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=1)
        )

    def forward(self, x):
        # x is len(session) x 3
        o, (h,c) = self.lstm(x) # o is shape (len(session), 64)
        o = nn.functional.relu(o[-1])
        logits = self.mlp(o)

        return logits

model = LSTM1().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [14]:
X,_ = load_one_session(data_dir, 0) 

# logits is the classification for the final point in X
logits = model(X.to(device).float())

# 2nd LSTM: Many sequences of points
- session is padded and windowed into `len(session)` windows
- each window is shaped into sequence of 101 3D inputs (xyz)
- input to LSTM is ($N$, $L$,$H_{in}$)

$N = 64$

$L = 101$

$H_{in} = 3$

In [15]:
class LSTM2(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=3, hidden_size=64, bias=False, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=1)
        )

    def forward(self, x):
        # x is batch_size x 303, want shape: batch_size x 101 x 3
        x = torch.cat(x.unsqueeze(2).split(WINSIZE, dim=1), axis=2)
        o, (h,c) = self.lstm(x) # o is shape (batch_size, winsize, 64)
        o = nn.functional.relu(o[:,-1,:])
        logits = self.mlp(o)

        return logits

model = LSTM2().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [16]:
dataset = load_one_windowed_session(data_dir, 0)
# X = dataset.tensors[0]
# X = torch.cat(X.unsqueeze(2).split(WINSIZE, dim=1), axis=2)
# dataset = TensorDataset(X, dataset.tensors[1])
loader = DataLoader(dataset, shuffle=True, batch_size=64)

# Not really accurate, since y here are labels for center of windows,
# and this model is predicted the final point in the window
train_loop(model, loader, criterion, optimizer, 10, device)

Starting train_loop


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 9: Train Loss: 0.0068326: 100%|██████████| 10/10 [00:08<00:00,  1.13it/s]

Finished train_loop. Elapsed time: 8.840


# 3rd LSTM: Many Sequences of Windows
- session is padded and windowed into `len(session)` windows
- each window is sequence of 303 inputs (flattened xyz)
- input to model is $(N,L, H_{in})$, $N=64$, $L=303$, $H_{in}=1$
- in model, each window is padded and windowed into 303 subwindows of length 15 (shape 303x15) before theyre fed to lstm
    - in window, first 101 are x, next 101 are y, etc.
    - in subwindows, first 5 are x, next 5 are z, etc.
- input to LSTM is ($N$, $L$, $H_{in}$)

$N = 64$

$L = 303$

$H_{in} = 15$

In [17]:
class LSTM3(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=15, hidden_size=64, bias=False, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=1)
        )
    
    def window_window(self, x, subwinsize):

        # Seperate x,y,z
        x = x.reshape(-1, 3, WINSIZE)

        # Pad x,y, and z on both sides with 0s
        x = nn.functional.pad(x, (subwinsize//2, subwinsize//2), 'constant', 0)

        # Window x,y, and z
        xacc = x[:,0].unsqueeze(2)
        yacc = x[:,1].unsqueeze(2)
        zacc = x[:,2].unsqueeze(2)

        w = subwinsize - 1
        xs = [xacc[:, :-w]]
        ys = [yacc[:, :-w]]
        zs = [zacc[:, :-w]]

        for i in range(1,w):
            xs.append(xacc[:, i:i-w])
            ys.append(yacc[:, i:i-w])
            zs.append(zacc[:, i:i-w])
        
        xs.append(xacc[:, w:])
        ys.append(yacc[:, w:])
        zs.append(zacc[:, w:])

        xs = torch.cat(xs, dim=2)
        ys = torch.cat(ys, dim=2)
        zs = torch.cat(zs, dim=2)

        # Re-combine each window of xyz
        return torch.cat([xs, ys, zs], dim=2)

    def forward(self, x):
        # x is batch_size x 303, want shape: batch_size x 101 x 15
        x = self.window_window(x, subwinsize=5)
        
        o, (h,c) = self.lstm(x) # o is shape (batch_size, winsize, 64)
        o = nn.functional.relu(o[:,-1,:])
        logits = self.mlp(o)

        return logits

model = LSTM3().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
ids = get_all_session_ids(data_dir)[:10]
train_dataset, dev_dataset, _ = nursingv1_train_dev_test_split(data_dir, 0.8, 0.2, 0, True, ids)

trainloader = DataLoader(train_dataset, shuffle=True, batch_size=64)
devloader = DataLoader(dev_dataset, shuffle=True, batch_size=64)

optimization_loop(model, trainloader, devloader, criterion, optimizer, 20, device)

# 4th LSTM: CNN -> LSTM
- input window to model of shape batch_size x 303
- model reshapes input to batch_size x 3 x 101 (three channels)
- uses CNN layer with 5 filters and padding to get output shape batch_size x 5 x 101
- input shape to CNN is $(N,C_{in},L_{in})$ and output shape is $(N,C_{out},L_{out})$, where:

$N=64$,  $C_{in}=3$, $L_{in}=101$, $C_{out}=5$, $L_{in}=101$

- This is reshaped to batch_size x 101 x 15 (transpose) and given to LSTM
- input to LSTM is $(N,L,H_{in})$, where:

$N=64$, $L=101$, $H_{in}=5$

- output of LSTM with 64 hidden states is batch_size x 101 x 64
- the final hidden state (shape batch_size x 64) is given to MLP to get one output

In [67]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super().__init__()

        self.c1 = nn.Conv1d(in_channels=3, out_channels=5, kernel_size=8, padding='same')
        self.l1 = nn.LSTM(input_size=5, hidden_size=64, bias=False, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=1)
        )
    
    def forward(self, x):
        # x is batch_size x 303, want shape: batch_size x 3 x 101
        x = x.view(-1, 3, WINSIZE)

        x = self.c1(x)
        x = torch.transpose(x, 1, 2)
        o, (h,c) = self.l1(x)
        o = nn.functional.relu(o[:,-1,:])
        logits = self.mlp(o)

        return logits

model = CNN_LSTM().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [69]:
ids = get_all_session_ids(data_dir)[:15]
train_dataset, dev_dataset, _ = nursingv1_train_dev_test_split(data_dir, 0.8, 0.2, 0, True, ids)

trainloader = DataLoader(train_dataset, shuffle=True, batch_size=64)
devloader = DataLoader(dev_dataset, shuffle=True, batch_size=64)

optimization_loop(model, trainloader, devloader, criterion, optimizer, 30, device)

Starting optimization_loop


Epoch 28: Train Loss: 0.0083876: Dev Loss: 0.24942:  97%|█████████▋| 29/30 [07:54<00:16, 16.42s/it]